In [32]:
# clear all variables from the notebook
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

import sympy as sp
import numpy as np

S_roots_planta = [-0.7547 + 1.1621*sp.I, -0.7547 - 1.1621*sp.I]

S_obs_factor = 10  # veces por arriba de los polos de la planta 



# Define the symbols
s = sp.symbols('s', complex=True)
S_roots_obs = []
for Si in S_roots_planta:
    S_roots_obs.append(Si * S_obs_factor)
    display(sp.Eq(sp.symbols(f'S_{{io}}'), sp.N(S_roots_obs[-1], 5)))

print(f"complex poles |S_io| = { np.abs(complex(sp.N(S_roots_obs[0], 5))):.03f}\n")

desired_obs_poly = 1
for root in S_roots_obs:
    desired_obs_poly *= (s - root)
desired_obs_poly = sp.N(desired_obs_poly, 5).as_poly(s).as_expr()

print("Desired observer polynomial:")
display(sp.Eq(sp.symbols('P_o'), desired_obs_poly, evaluate=False))

Eq(S_{io}, -7.547 + 11.621*I)

Eq(S_{io}, -7.547 - 11.621*I)

complex poles |S_io| = 13.857

Desired observer polynomial:


Eq(P_o, 1.0*s**2 + 15.094*s + 192.0)

In [33]:
# Define A, B, C matrices
A = [   [0,     1       ], 
        [0,     -261.72 ]]

B = [   [0      ], 
        [244.68 ]]

C = [   [1,     0]  ]


# Ya definieron todo lo necesario para calcular, ahora es solo ver los resultados
A = sp.Matrix(A)
B = sp.Matrix(B)
C = sp.Matrix(C)

A_sym = sp.MatrixSymbol('A', 2, 2)
B_sym = sp.MatrixSymbol('B', 2, 1)
C_sym = sp.MatrixSymbol('C', 1, 2)
L1, L2 = sp.symbols('L1 L2')

L = sp.Matrix([[L1], [L2]])


L_eq = sp.Eq( sp.Determinant(s * sp.Identity(2) - sp.MatAdd(A_sym, -sp.MatMul(L,C_sym, evaluate=False), evaluate=False)), 0, evaluate=False)
print("L equation:")
display(L_eq)

L_eq = L_eq.subs(A_sym, A).subs(B_sym, B).subs(C_sym, C).subs(sp.Identity(2), sp.eye(2)).expand().doit()
print("with values")
display(L_eq)
# # combine the terms

L_eq = sp.Eq(sp.Determinant(L_eq.lhs.arg.doit()), 0)
print("combined")
display(L_eq)

observer_poly = L_eq.doit().as_poly(s).as_expr()
print("simplified")
display(sp.Eq(observer_poly, 0))

L equation:


Eq(Determinant(s*I + Matrix([
[L1],
[L2]])*C - A), 0)

with values


Eq(Determinant(Matrix([
[0,     -1],
[0, 261.72]]) + Matrix([
[L1, 0],
[L2, 0]]) + Matrix([
[s, 0],
[0, s]])), 0)

combined


Eq(Determinant(Matrix([
[L1 + s,         -1],
[    L2, s + 261.72]])), 0)

simplified


Eq(261.72*L1 + 1.0*L2 + 1.0*s**2 + s*(1.0*L1 + 261.72), 0)

In [34]:
# iterate both polynomials to get the coefficients
eqs = []
for i in range(3):
    lhs = observer_poly.as_poly(s).coeffs()[i]
    rhs = desired_obs_poly.as_poly(s).coeffs()[i]
    eqs.append(sp.Eq(lhs, rhs))

sol = sp.solve(eqs, (L1, L2))

print("Solution:")
for key, val in sol.items():
    display(sp.Eq(key, sp.N(val, 4)))

Solution:


Eq(L1, -246.6)

Eq(L2, 6.474e+4)